In [1]:
import pandas
from pandas import DataFrame
from typing import Tuple


# Cargar los datos del archivo CSV en un tipo de datos complejo, como listas, diccionarios, conjuntos o combinación de ellos (ej. un diccionario de listas)
dominios_dataframe: DataFrame = pandas.read_csv('https://www.nic.cl/estadisticas/numDominios.csv')
dominios_dataframe.replace("", pandas.NA, inplace=True)

In [2]:
# Reportar el mes y el año con mayor cantidad de dominios **.CL**
valor_maximo: float = dominios_dataframe.iloc[:, 1:].max().max()
posicion_maximo: Tuple[int, str] = dominios_dataframe.iloc[:, 1:].stack().idxmax()
mes_maximo: str = posicion_maximo[1]
año_maximo: int = dominios_dataframe.loc[posicion_maximo[0], 'Año']
print(f"Valor Máximo: {int(valor_maximo)}\nMes: {mes_maximo}\nAño: {año_maximo}")



Valor Máximo: 736931
Mes: Octubre
Año: 2021


In [3]:
# Reportar los dos meses consecutivos entre los cuales se encuentra la mayor variación (en números totales, no porcentual) en la cantidad de dominios **.CL**
variacion: DataFrame = dominios_dataframe.iloc[:, 1:].diff(axis=1).iloc[:, 1:].abs()
posicion_diferencia_maxima: Tuple[int, str] = variacion.stack().idxmax()
diferencia_maxima: float = variacion.max().max()
indice_fila_maximo: int = posicion_diferencia_maxima[0]
mes_maximo: str = posicion_diferencia_maxima[1]
lista_meses: list[str] = variacion.columns
indice_mes_maximo: int = lista_meses.get_loc(mes_maximo)
año_maximo: int = dominios_dataframe.iloc[indice_fila_maximo]["Año"]
if indice_mes_maximo > 0:
    mes_previo: str = lista_meses[indice_mes_maximo - 1]
    mes1: str = mes_previo
    mes2: str = mes_maximo
else:
    mes1: str = None
    mes2: str = mes_maximo
print(f"La mayor variación de dominios es {int(diferencia_maxima)} entre los meses {mes1} y {mes2} del año {int(año_maximo)}")

La mayor variación de dominios es 19117 entre los meses Abril y Mayo del año 2020


In [13]:
# Crear un archivo CSV que filtre sólo los meses con al menos 500 mil dominios
dataframe_filtrado = dominios_dataframe.map(lambda x: int(x) if (isinstance(x, (int, float)) and x >= 500000) else pandas.NA)
dataframe_filtrado.to_csv('numDominiosFiltrado.csv', index=False)

C:\Users\ivill\AppData\Local\Temp\ipykernel_25936\885584582.py:2: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [9]:
import plotly.express as px

traduccion_meses: dict[str, str] = {
    "Enero": "Jan", "Febrero": "Feb", "Marzo": "Mar", "Abril": "Apr",
    "Mayo": "May", "Junio": "Jun", "Julio": "Jul", "Agosto": "Aug",
    "Sept.": "Sep", "Octubre": "Oct", "Noviem.": "Nov", "Diciem.": "Dec"
}
dataframe_mergeado: DataFrame = dominios_dataframe.melt(id_vars=["Año"], var_name="Mes", value_name="Número de dominios")
dataframe_mergeado["Mes"] = dataframe_mergeado["Mes"].map(traduccion_meses)
dataframe_mergeado["Fecha"] = pandas.to_datetime(dataframe_mergeado["Año"].astype(str) + '-' + dataframe_mergeado["Mes"], format='%Y-%b')
dataframe_mergeado: DataFrame = dataframe_mergeado.sort_values(by="Fecha")
fig = px.line(dataframe_mergeado, x="Fecha", y="Número de dominios",
              title="Número de dominios comprados desde enero de 1997 hasta diciembre de 2024")
fig.show()